In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns',50)

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train=pd.read_csv('/kaggle/input/daconcard/open/train.csv')
test=pd.read_csv('/kaggle/input/daconcard/open/test.csv')
submit=pd.read_csv('/kaggle/input/daconcard/open/sample_submission.csv')

In [ ]:
def datainfo(df):
    return pd.DataFrame([(col,df[col].dtype,df[col].isna().sum(),df[col].nunique(),df[col].unique()[:5]) for col in df.columns],
                       columns=['name','dtype','missing','nunique','values :5'])

In [ ]:
datainfo(train)

In [ ]:
## ordinal : edu_type(Lower,Secondary,Incomplete,Higher,academ),child_num() income(Student,)
## binary : gender,car,reality,work_phone,phone,email,
## nominal : 

In [ ]:
datainfo(test)

In [ ]:
datainfo(submit)

In [ ]:
## 제거 
train=train.drop(['FLAG_MOBIL','index'],axis=1)
test=test.drop(['FLAG_MOBIL','index'],axis=1)

## 중복 확인 및 제거
cols=['gender', 'car', 'reality', 'child_num', 'income_total',
      'income_type', 'edu_type', 'family_type', 'house_type', 'DAYS_BIRTH',
      'DAYS_EMPLOYED','work_phone', 'phone', 'email',
      'occyp_type', 'family_size']

## 마지막에 입력된 데이터가 바뀌어도 진짜라고 가정. drop 안했을 때 성능 굳
# train=train.drop_duplicates(subset=cols,keep='last')

## 고용일 처리 -> 양수 성능 굳 ? 
# train['DAYS_BIRTH']=train['DAYS_BIRTH']*(-1/365)
# test['DAYS_BIRTH']=test['DAYS_BIRTH']*(-1/365)

# train['DAYS_EMPLOYED']=train['DAYS_EMPLOYED']*(-1/365)
# train.loc[train['DAYS_EMPLOYED']<0,'DAYS_EMPLOYED']=-999
# test['DAYS_EMPLOYED']=test['DAYS_EMPLOYED']*(-1/365)
# test.loc[test['DAYS_EMPLOYED']<0,'DAYS_EMPLOYED']=-999

## 가족수 이상치 나이대로 판별 이상 판단.
train=train[(train['family_size']!=20)&(train['family_size']!=15)]
# 결측값 
train['occyp_type']=train['occyp_type'].fillna('unknown')
test['occyp_type']=test['occyp_type'].fillna('unknown')

In [ ]:
train['DAYS_BIRTH']=train['DAYS_BIRTH']*(-1/365)
test['DAYS_BIRTH']=test['DAYS_BIRTH']*(-1/365)

train['DAYS_BIRTH']=pd.cut(train['DAYS_BIRTH'],bins=[20,30,40,50,60,70],labels=['20s','30s','40s','50s','60s'])

train['DAYS_EMPLOYED']=train['DAYS_EMPLOYED']*(-1/365)
# train.loc[train['DAYS_EMPLOYED']<0,'DAYS_EMPLOYED']=-999
test['DAYS_EMPLOYED']=test['DAYS_EMPLOYED']*(-1/365)
# test.loc[test['DAYS_EMPLOYED']<0,'DAYS_EMPLOYED']=-999

train['DAYS_EMPLOYED']=pd.cut(train['DAYS_EMPLOYED'],bins=[-10000,0,1,5,10,20,10000],labels=['nan','1y','5y','10y','20y','>20y'])

In [ ]:
birth_order = {
    '<20' : 0, 
    '20s' : 1, 
    '30s' : 2,
    '40s' : 3, 
    '50s' : 4 ,
    '60s':5,
}
train.DAYS_BIRTH = train.DAYS_BIRTH.map(birth_order)
test.DAYS_BIRTH = test.DAYS_BIRTH.map(birth_order)

In [ ]:
emp_order = {
    'nan' : 0, 
    '1y' : 1, 
    '5y' : 2,
    '10y' : 3, 
    '20y' : 4,
    '>20y':5
}
train.DAYS_EMPLOYED = train.DAYS_EMPLOYED.map(emp_order)
test.DAYS_EMPLOYED = test.DAYS_EMPLOYED.map(emp_order)


In [ ]:
edu_order = {
    'Lower secondary' : 0, 
    'Secondary / secondary special' : 1, 
    'Incomplete higher' : 2,
    'Higher education' : 3, 
    'Academic degree' : 4 
}
train.edu_type = train.edu_type.map(edu_order)
test.edu_type = test.edu_type.map(edu_order)

# train.groupby('income_type')['income_total'].median()

income_order={
    'Pensioner':0,
    'Working':1,
    'Student':2,
    'State servant':3,
    'Commercial associate':4
}
train.income_type=train.income_type.map(income_order)
test.income_type=test.income_type.map(income_order)

# train.groupby('family_type')['income_total'].median()

family_order={
    'Civil marriage':0,
    'Separated':0,
    'Married':1,
    'Widow':1,
    'Single / not married':2
}
train.family_type=train.family_type.map(family_order)
test.family_type=test.family_type.map(family_order)

# train.groupby('house_type')['income_total'].median()

house_order={
    'House / apartment':0,
    'Municipal apartment':0,
    'With parents':0,
    'Rented apartment':1,
    'Office apartment':1,
    'Co-op apartment':2
}
train.house_type=train.house_type.map(house_order)
test.house_type=test.house_type.map(house_order)

# train.groupby('occyp_type')['income_total'].median()
## co-op 는 가격의 차이가 있다는 가정.

occyp_order={
    'unknown':0,
    'Low-skill Laborers':1,
    'Secretaries':2,
    'Cleaning staff':2,
    'Cooking staff':2,
    'HR staff':3,
    'Laborers':3,
    'Medicine staff':3,
    'Sales staff':3,
    'Security staff':3,
    'Waiters/barmen staff':3,
    'Private service staff':4,
    'High skill tech staff':4,
    'Accountants':4,
    'Core staff':4,
    'IT staff':4,
    'Realty agents':5,
    'Drivers':5,
    'Managers':5
}
train.occyp_type=train.occyp_type.map(occyp_order)
test.occyp_type=test.occyp_type.map(occyp_order)

train['gender']=train['gender'].apply(lambda x:1 if x=='M' else 0)
train['car']=train['car'].apply(lambda x:1 if x=='N' else 0)
train['reality']=train['reality'].apply(lambda x:1 if x=='N' else 0)

test['gender']=test['gender'].apply(lambda x:1 if x=='M' else 0)
test['car']=test['car'].apply(lambda x:1 if x=='N' else 0)
test['reality']=test['reality'].apply(lambda x:1 if x=='N' else 0)

In [ ]:
train['begin_month']*=(-1/12)
test['begin_month']*=(-1/12)

train['begin_month']=pd.cut(train['begin_month'],bins=[-1,0.5,1.1,2.1,3.1,4.1,5.1],labels=['0y','1/2y','1y','2y','3y','4y'])
test['begin_month']=pd.cut(test['begin_month'],bins=[-1,0.5,1.1,2.1,3.1,4.1,5.1],labels=['0y','1/2y','1y','2y','3y','4y'])

In [ ]:
begin_order={
    '0y':0,
    '1/2y':1,
    '1y':2,
    '2y':3,
    '3y':4,
    '4y':5
}
train.begin_month=train.begin_month.map(begin_order)
test.begin_month=test.begin_month.map(begin_order)

train['begin_order']=train['begin_month'].astype(int)
test['begin_order']=test['begin_month'].astype(int)

In [ ]:
## 시도 파생변수 : 성능 더 떨어진다.

# train['family_size']=pd.Categorical(train['family_size'],ordered=True)
# train['child_num']=pd.Categorical(train['child_num'],ordered=True)
# train['begin_month']=pd.Categorical(train['begin_month'],ordered=True)

# test['family_size']=pd.Categorical(test['family_size'],ordered=True)
# test['child_num']=pd.Categorical(test['child_num'],ordered=True)
# test['begin_month']=pd.Categorical(test['begin_month'],ordered=True)

In [ ]:
## income과 gender에 따라 차이가 없다.
# sns.catplot(
#     data=train, kind="bar",
#     x="credit", y="income_total", hue="gender",
#     ci="sd", palette="dark", alpha=.6, height=6
# )

## car reality에 따라 income의 차이는 있다.
# sns.catplot(
#     data=train, kind="bar",
#     x="car", y="income_total", hue="reality",
#     ci="sd", palette="dark", alpha=.6, height=6
# )

## car reality도 차이없다. 
# sns.catplot(
#     data=train, kind="bar",
#     x="credit", y="income_total", hue="reality",
#     ci="sd", palette="dark", alpha=.6, height=6
# )

## income_total은 제거 해본다.
# sns.catplot(
#     data=train, kind="bar",
#     x="credit", y="income_total", hue="begin_month",
#     ci="sd", palette="dark", alpha=.6, height=6
# )

In [ ]:
## #했을때 비슷.
# train['gender']=train['gender'].apply(lambda x:1 if x=='M' else 0)
# train['car']=train['car'].apply(lambda x:1 if x=='N' else 0)
# train['reality']=train['reality'].apply(lambda x:1 if x=='N' else 0)
# train['work_phone']=train['work_phone'].apply(lambda x:1 if x==0 else 0)
# train['phone']=train['phone'].apply(lambda x:1 if x==0 else 0)
# train['email']=train['email'].apply(lambda x:1 if x==0 else 0)

# test['gender']=test['gender'].apply(lambda x:1 if x=='M' else 0)
# test['car']=test['car'].apply(lambda x:1 if x=='N' else 0)
# test['reality']=test['reality'].apply(lambda x:1 if x=='N' else 0)
# test['work_phone']=test['work_phone'].apply(lambda x:1 if x==0 else 0)
# test['phone']=test['phone'].apply(lambda x:1 if x==0 else 0)
# test['email']=test['email'].apply(lambda x:1 if x==0 else 0)

In [ ]:
# tr_cat_dum=pd.get_dummies(train[['income_type','edu_type','family_type','house_type','occyp_type']],drop_first=True)
# ts_cat_dum=pd.get_dummies(test[['income_type','edu_type','family_type','house_type','occyp_type']],drop_first=True)

In [ ]:
# train=train.drop(['income_type','edu_type','family_type','house_type','occyp_type'],axis=1)
# test=test.drop(['income_type','edu_type','family_type','house_type','occyp_type'],axis=1)

In [ ]:
# train=pd.concat([train,tr_cat_dum],axis=1)
# test=pd.concat([test,ts_cat_dum],axis=1)

In [ ]:
## label
## 수가 많아서 이걸로.

# from category_encoders.ordinal import OrdinalEncoder

# oe=OrdinalEncoder(handle_missing=-999)

# oe.fit(train['family#begin'])
# train['family#begin']=oe.transform(train['family#begin'])
# test['family#begin']=oe.transform(test['family#begin'])

In [ ]:
## 가설1. 다중공선성
## 가설2. income_total 구분.
# train=train.drop(['DAYS_BIRTH','child_num'],axis=1)
# test=test.drop(['DAYS_BIRTH','child_num'],axis=1)

# train['income_total']=pd.cut(train['income_total'],bins=[0,100000,350000,500000,750000,1000000,2000000],labels=['F','E','D','C','B','A'])
# test['income_total']=pd.cut(test['income_total'],bins=[0,100000,350000,500000,750000,1000000,2000000],labels=['F','E','D','C','B','A'])\

In [ ]:
## feature importance에는 좋다고 나오는데 성능은 1.2 뭐가 문제일까.
## 1. + - 실험. 과적합
## 2. * 실험 뭔가 맞춰지는. 다른것과 시너지가 있는것 같다.
train['child*family']=train.apply(lambda row:row['child_num']*row['family_size'],axis=1)
test['child*family']=test.apply(lambda row:row['child_num']*row['family_size'],axis=1)

train=train.drop(['family_size','child_num'],axis=1)
test=test.drop(['family_size','child_num'],axis=1)

In [ ]:
cat_cols=['gender','car','reality','income_type','edu_type','family_type',
          'house_type','work_phone','phone','email','occyp_type','child*family',
          'DAYS_EMPLOYED','DAYS_BIRTH']

from itertools import combinations as comb

for a,b in comb(cat_cols,2):
    df=train.groupby([a,b,'begin_month'],as_index=False)['income_total'].mean()
    df.columns=[a,b,'begin_month',f'{a}_{b}_grpmean']
    train=pd.merge(train,df,how='left',on=[a,b,'begin_month'])
    test=pd.merge(test,df,how='left',on=[a,b,'begin_month'])

In [ ]:
drop_cols=[]
for col in test.columns:
    if test[col].isnull().sum():
        drop_cols.append(col)

In [ ]:
train=train.drop(drop_cols,axis=1)
test=test.drop(drop_cols,axis=1)

In [ ]:
# train=pd.get_dummies(train,drop_first=True)
# test=pd.get_dummies(test,drop_first=True)

In [ ]:
train.shape,test.shape

In [ ]:
train['begin_month']=train['begin_month'].astype(int)

In [ ]:
X=train.drop('credit',axis=1)
y=train['credit']

from sklearn.preprocessing import normalize

X=normalize(X,axis=1)
testc=test.copy()
testc=normalize(testc,axis=1)

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.15,random_state=71,stratify=y)

## xgb

In [ ]:
import xgboost as xgb

params={
    'objective':'multi:softprob',
    'random_state':71,
    'n_estimators':1000
}
model=xgb.XGBClassifier(**params)

model.fit(X_train,y_train,
          eval_set=[(X_train, y_train),(X_test,y_test)],
          eval_metric='mlogloss',verbose=True,
          early_stopping_rounds=10)

In [ ]:
pred=pd.DataFrame(model.predict_proba(testc))
pred['index']=submit['index']
pred=pred[['index',0,1,2]]
pred.head()

In [ ]:
xgb.plot_importance(model,max_num_features=10)

In [ ]:
xgb.to_graphviz(model)

In [ ]:
pred.to_csv('20210508_xgboost_all_cat_grpincome.csv',index=False)

In [ ]:
## income_total 정규성 x -> log 변환 한것을 파생변수로.
## 다른건 몰라도 income_type이 pensioner인 사람들이 DAYS_BIRTH가 오래됐다. 
## -> DAYS_BIRTH는 -7500정도에서 -25000정도 사이 나이로 바꿀 필요가 있을까? -> 보류
## edu_type에 따라 income_total과 DAYS_BIRTH는 선형관계가 있다. 
## DAYS_EMPLOYED 양수 값은 고용되지 않은 상태를 의미함 -> 제거 
## family type과 성별에 따라 income_total의 차이가 없다. 
## FLAG_MOBIL 1밖에 없다. -> 제거 
## 중복값 제거
## family_size child_num 상관관계 -> 제거,income_total income_total_log 제거 그 외 그냥 둔다.

sns.jointplot(data=train, x="family_size", y="begin_month", hue="house_type")
# sns.displot(data=train, x="income_total", col="DAYS_BIRTH", kde=True)
# sns.relplot(data=train,x='income_total',y='DAYS_BIRTH',hue='edu_tpe')

In [ ]:
## 연속형 - 연속
sns.relplot(
    data=train,
    x="income_total", y="tip", col="time",
    hue="smoker", style="smoker", size="size",
)

sns.lmplot(data=tips, x="total_bill", y="tip", col="time", hue="smoker")
sns.displot(data=tips, x="total_bill", col="time", kde=True)
sns.jointplot(data=penguins, x="flipper_length_mm", y="bill_length_mm", hue="species")

In [ ]:
sns.catplot(data=train[train['income_total']>=],kind='violin',x='credit',y='income_total')

In [ ]:
sns.catplot(data=train, kind="violin", x="gender", y="income_total_log", hue="family_type",split=False)

In [ ]:
## 범주 - 연속
sns.catplot(data=tips, kind="swarm", x="day", y="total_bill", hue="smoker",split=True)